<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 28.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torch.optim import SGD

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv(path+"/train.csv")
validation = pd.read_csv(path+"/validation.csv")
test = pd.read_csv(path+"/boneage-test-dataset.csv")

In [9]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 136.62 KiB | 5.06 MiB/s, done.


In [10]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

In [11]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, data, image_dir):
        self.image_dir = image_dir
        #self.pickle_file = pickle_file
        #self.frame = frame
        self.data = data
        self.table = pd.DataFrame(data)


    def __len__(self):
        return len(self.table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        table = self.table.iloc[idx, 0:]

        y = table["boneage"]

        image = Image.open(f"{self.image_dir}/{table['id']}.png")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        table = table[["male"]]
        table = table.tolist()
        table = torch.FloatTensor(table)

        return image, table, y

In [12]:
class BoNetModule(pl.LightningModule):
    def __init__(self, transform_input = False, lr = 0.0001, num_workers = 0, batch_size = 20
    ):
        # Define Model Here
        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x,y):
        # Define Forward Pass Here
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x


    def configure_optimizers(self):
       # Define Optimizer Here
       return torch.optim.Adam(self.parameters(), lr=(self.lr),)

    def training_step(self, batch, batch_idx):
      image, table, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, table))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}       


    def setup(self, stage):

        image_datatrain = ImageDataset(train, image_dir=f"{train_img}")
        image_dataval = ImageDataset(validation, image_dir=f"{val_img}")
        image_datatest = ImageDataset(test, image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)



In [13]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import nn, optim, rand, sum as tsum, reshape, save
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint

In [14]:
logger = pl.loggers.TensorBoardLogger("lightning_logs",version = 'version_0', name="my_model")
checkpoint_callback = ModelCheckpoint(dirpath = "lightning_logs",verbose=True, mode='min',every_n_train_steps = None, every_n_epochs = None)

model = BoNetModule()


trainer = pl.Trainer(accelerator='gpu',devices=1, logger=logger,max_epochs=5, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,callbacks=[checkpoint_callback])
trainer.fit(model)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type        | Params
---------------------------------------------
0 | inception_v3 | InceptionV3 | 21.8 M
1 | gender       | DenseLayer  | 64    
2 | fc_1         | DenseLayer  | 100 M 
3 | fc_2         | DenseLayer  | 1.0 M 
4 | fc_3         | Linear      | 1.0 K 
---------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     Total params
492.688   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

<ipython-input-11-b32ef9e936cb>:38: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  table = torch.FloatTensor(table)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
